<a href="https://colab.research.google.com/github/cchang-vassar/Semantic-Relations-in-Vector-Embeddings/blob/main/study2_9_%5Bnomic%5D_scifact_autoencoder_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [nomic] Autoencoder: Generate Corresponding Embedding (Scifact)

## Set Up

### Imports

In [ ]:
!pip install tensorflow

In [ ]:
!pip install plotnine

In [ ]:
import os
import subprocess
import zipfile
import shutil
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
from google.colab import userdata
from scipy import spatial
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from plotnine import ggplot, geom_line, aes, ggsave, labs, theme, element_text, guides, guide_legend, scale_color_manual, scale_y_continuous, scale_linetype_manual

### OSF Setup

In [ ]:
!pip install osfclient

In [ ]:
import osfclient.cli

In [ ]:
from osfclient.api import OSF
from osfclient.models import Project, Storage
from io import BytesIO

In [ ]:
os.environ["OSF_USERNAME"] = userdata.get("OSF_USERNAME")
OSF_USERNAME = os.environ["OSF_USERNAME"]

In [ ]:
os.environ["OSF_PASSWORD"] = userdata.get("OSF_PASSWORD")
OSF_PASSWORD = os.environ["OSF_PASSWORD"]

In [ ]:
os.environ["OSF_TOKEN"] = userdata.get("OSF_TOKEN")
OSF_TOKEN = os.environ["OSF_TOKEN"]

In [ ]:
os.environ["OSF_PROJECT_ID"] = userdata.get("OSF_PROJECT_ID")
OSF_PROJECT_ID = os.environ["OSF_PROJECT_ID"]

## Data

### Import corpora data from OSF

In [ ]:
!osf -p sakjg fetch osfstorage/data-dump/nomic-autoencoder/nomic_scifact_embeddings_dump.zip

100% 7.96M/7.96M [00:00<00:00, 49.0Mbytes/s]


In [ ]:
embeddings_file_path = 'nomic_scifact_embeddings_dump.zip'
output_folder_path = 'current-data-dump/embeddings-dump'
os.makedirs(output_folder_path, exist_ok=True)

with zipfile.ZipFile(embeddings_file_path, 'r') as zip_ref:
  zip_ref.extractall(output_folder_path)

extracted_files = os.listdir(output_folder_path)
print("Files extracted:", extracted_files)

Files extracted: ['training_pro_argument_embeddings.pkl', 'combined_counter_evidence_embeddings.pkl', 'training_pro_evidence_embeddings.pkl', 'combined_pro_argument_embeddings.pkl', 'test_counter_evidence_embeddings.pkl', 'training_counter_argument_embeddings.pkl', 'test_counter_argument_embeddings.pkl', 'combined_counter_argument_embeddings.pkl', 'combined_pro_evidence_embeddings.pkl', 'test_pro_evidence_embeddings.pkl', 'test_pro_argument_embeddings.pkl', 'training_counter_evidence_embeddings.pkl']


In [ ]:
x_train_pro = pd.read_pickle('current-data-dump/embeddings-dump/training_pro_argument_embeddings.pkl')
y_train_pro = pd.read_pickle('current-data-dump/embeddings-dump/training_pro_evidence_embeddings.pkl')
x_test_pro = pd.read_pickle('current-data-dump/embeddings-dump/test_pro_argument_embeddings.pkl')
y_test_pro = pd.read_pickle('current-data-dump/embeddings-dump/test_pro_evidence_embeddings.pkl')

In [ ]:
x_train_counter = pd.read_pickle('current-data-dump/embeddings-dump/training_counter_argument_embeddings.pkl')
y_train_counter = pd.read_pickle('current-data-dump/embeddings-dump/training_counter_evidence_embeddings.pkl')
x_test_counter = pd.read_pickle('current-data-dump/embeddings-dump/test_counter_argument_embeddings.pkl')
y_test_counter = pd.read_pickle('current-data-dump/embeddings-dump/test_counter_evidence_embeddings.pkl')

In [ ]:
x_train_pro = x_train_pro.drop(columns=['argument'])
y_train_pro = y_train_pro.drop(columns=['evidence'])
x_test_pro = x_test_pro.drop(columns=['argument'])
y_test_pro = y_test_pro.drop(columns=['evidence'])

In [ ]:
x_train_counter = x_train_counter.drop(columns=['argument'])
y_train_counter = y_train_counter.drop(columns=['evidence'])
x_test_counter = x_test_counter.drop(columns=['argument'])
y_test_counter = y_test_counter.drop(columns=['evidence'])

In [ ]:
x_train_combined = pd.concat([x_train_pro, x_train_counter])
y_train_combined = pd.concat([y_train_pro, y_train_counter])
x_test_combined = pd.concat([x_test_pro, x_test_counter])
y_test_combined = pd.concat([y_test_pro, y_test_counter])

### Make global data shuffled

In [ ]:
y_train_shuffled = y_train_combined.copy().sample(frac=1).reset_index(drop=True)
y_train_pro_shuffled = y_train_pro.copy().sample(frac=1).reset_index(drop=True)
y_train_counter_shuffled = y_train_counter.copy().sample(frac=1).reset_index(drop=True)

### Save global training df

In [ ]:
x_train_combined_folder_path = 'current-data-dump/nomic-autoencoder/'
x_train_combined_file_path = f'{x_train_combined_folder_path}x_train_combined.pkl'
y_train_combined_folder_path = 'current-data-dump/nomic-autoencoder/'
y_train_combined_file_path = f'{x_train_combined_folder_path}y_train_combined.pkl'
os.makedirs(x_train_combined_folder_path, exist_ok=True)
os.makedirs(y_train_combined_folder_path, exist_ok=True)
with open(x_train_combined_file_path, 'wb') as file:
  pickle.dump(x_train_combined, file)
  print(f"File uploaded to {x_train_combined_file_path}")
with open(y_train_combined_file_path, 'wb') as file:
  pickle.dump(y_train_combined, file)
  print(f"File uploaded to {y_train_combined_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_train_combined.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_train_combined.pkl


In [ ]:
x_test_combined_folder_path = 'current-data-dump/nomic-autoencoder/'
x_test_combined_file_path = f'{x_test_combined_folder_path}x_test_combined.pkl'
y_test_combined_folder_path = 'current-data-dump/nomic-autoencoder/'
y_test_combined_file_path = f'{x_test_combined_folder_path}y_test_combined.pkl'
os.makedirs(x_test_combined_folder_path, exist_ok=True)
os.makedirs(y_test_combined_folder_path, exist_ok=True)
with open(x_test_combined_file_path, 'wb') as file:
  pickle.dump(x_test_combined, file)
  print(f"File uploaded to {x_test_combined_file_path}")
with open(y_test_combined_file_path, 'wb') as file:
  pickle.dump(y_test_combined, file)
  print(f"File uploaded to {y_test_combined_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_test_combined.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_test_combined.pkl


In [ ]:
x_train_pro_folder_path = 'current-data-dump/nomic-autoencoder/'
x_train_pro_file_path = f'{x_train_pro_folder_path}x_train_pro.pkl'
y_train_pro_folder_path = 'current-data-dump/nomic-autoencoder/'
y_train_pro_file_path = f'{x_train_pro_folder_path}y_train_pro.pkl'
os.makedirs(x_train_pro_folder_path, exist_ok=True)
os.makedirs(y_train_pro_folder_path, exist_ok=True)
with open(x_train_pro_file_path, 'wb') as file:
  pickle.dump(x_train_pro, file)
  print(f"File uploaded to {x_train_pro_file_path}")
with open(y_train_pro_file_path, 'wb') as file:
  pickle.dump(y_train_pro, file)
  print(f"File uploaded to {y_train_pro_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_train_pro.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_train_pro.pkl


In [ ]:
x_test_pro_folder_path = 'current-data-dump/nomic-autoencoder/'
x_test_pro_file_path = f'{x_test_pro_folder_path}x_test_pro.pkl'
y_test_pro_folder_path = 'current-data-dump/nomic-autoencoder/'
y_test_pro_file_path = f'{x_test_pro_folder_path}y_test_pro.pkl'
os.makedirs(x_test_pro_folder_path, exist_ok=True)
os.makedirs(y_test_pro_folder_path, exist_ok=True)
with open(x_test_pro_file_path, 'wb') as file:
  pickle.dump(x_test_pro, file)
  print(f"File uploaded to {x_test_pro_file_path}")
with open(y_test_pro_file_path, 'wb') as file:
  pickle.dump(y_test_pro, file)
  print(f"File uploaded to {y_test_pro_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_test_pro.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_test_pro.pkl


In [ ]:
x_train_counter_folder_path = 'current-data-dump/nomic-autoencoder/'
x_train_counter_file_path = f'{x_train_counter_folder_path}x_train_counter.pkl'
y_train_counter_folder_path = 'current-data-dump/nomic-autoencoder/'
y_train_counter_file_path = f'{x_train_counter_folder_path}y_train_counter.pkl'
os.makedirs(x_train_counter_folder_path, exist_ok=True)
os.makedirs(y_train_counter_folder_path, exist_ok=True)
with open(x_train_counter_file_path, 'wb') as file:
  pickle.dump(x_train_counter, file)
  print(f"File uploaded to {x_train_counter_file_path}")
with open(y_train_counter_file_path, 'wb') as file:
  pickle.dump(y_train_counter, file)
  print(f"File uploaded to {y_train_counter_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_train_counter.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_train_counter.pkl


In [ ]:
x_test_counter_folder_path = 'current-data-dump/nomic-autoencoder/'
x_test_counter_file_path = f'{x_test_counter_folder_path}x_test_counter.pkl'
y_test_counter_folder_path = 'current-data-dump/nomic-autoencoder/'
y_test_counter_file_path = f'{x_test_counter_folder_path}y_test_counter.pkl'
os.makedirs(x_test_counter_folder_path, exist_ok=True)
os.makedirs(y_test_counter_folder_path, exist_ok=True)
with open(x_test_counter_file_path, 'wb') as file:
  pickle.dump(x_test_counter, file)
  print(f"File uploaded to {x_test_counter_file_path}")
with open(y_test_counter_file_path, 'wb') as file:
  pickle.dump(y_test_counter, file)
  print(f"File uploaded to {y_test_counter_file_path}")

File uploaded to current-data-dump/nomic-autoencoder/x_test_counter.pkl
File uploaded to current-data-dump/nomic-autoencoder/y_test_counter.pkl


## Model

### Architecture

In [ ]:
# Layers
input_layer = tf.keras.layers.Input(shape=(768, ), name="Input")
hidden_layer = tf.keras.layers.Dense(units=768, activation="relu", name="Hidden")(input_layer)
output_layer = tf.keras.layers.Dense(units=768, activation="linear", name="Output")(hidden_layer)

In [ ]:
# Model
autoencoder_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
autoencoder_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 768)]             0         
                                                                 
 Hidden (Dense)              (None, 768)               590592    
                                                                 
 Output (Dense)              (None, 768)               590592    
                                                                 
Total params: 1181184 (4.51 MB)
Trainable params: 1181184 (4.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Metric

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_combined_y_train(y_true, y_pred):
  """combined_metric"""
  combined_training_df_32 = tf.cast(pd.concat([x_train_combined, y_train_combined, x_test_combined, y_test_combined]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(combined_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(combined_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(combined_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(combined_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_pro_y_train(y_true, y_pred):
  """pro_metric"""
  pro_training_df_32 = tf.cast(pd.concat([x_train_pro, y_train_pro, x_test_pro, y_test_pro]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(pro_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(pro_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(pro_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(pro_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_counter_y_train(y_true, y_pred):
  """counter_metric"""
  counter_training_df_32 = tf.cast(pd.concat([x_train_counter, y_train_counter, x_test_counter, y_test_counter]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(counter_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(counter_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(counter_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(counter_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

### Combined Training

In [ ]:
# combined Model
combined_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
combined_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_combined_y_train]
)

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='combined_autoencoder_weights.keras', save_best_only=False, save_weights_only=False, verbose=1)
csv_logger_callback = CSVLogger(filename='combined_training_log.csv', separator=',', append=True)
combined_history = combined_autoencoder_model.fit(
  x=x_train_combined,
  y=y_train_combined,
  batch_size=1,
  epochs=100,
  validation_data = (x_test_combined, y_test_combined),
  callbacks=[checkpoint_callback, csv_logger_callback]
)

Epoch 1/100
493/505 [============================>.] - ETA: 0s - loss: -0.7825 - metric_choose_argument_combined_y_train: 0.0122
Epoch 1: saving model to combined_autoencoder_weights.keras
505/505 [==============================] - 7s 4ms/step - loss: -0.7831 - metric_choose_argument_combined_y_train: 0.0139 - val_loss: -0.8049 - val_metric_choose_argument_combined_y_train: 0.0479
Epoch 2/100
496/505 [============================>.] - ETA: 0s - loss: -0.8274 - metric_choose_argument_combined_y_train: 0.2117
Epoch 2: saving model to combined_autoencoder_weights.keras
505/505 [==============================] - 2s 4ms/step - loss: -0.8274 - metric_choose_argument_combined_y_train: 0.2139 - val_loss: -0.8212 - val_metric_choose_argument_combined_y_train: 0.1543
Epoch 3/100
492/505 [============================>.] - ETA: 0s - loss: -0.8584 - metric_choose_argument_combined_y_train: 0.4736
Epoch 3: saving model to combined_autoencoder_weights.keras
505/505 [==============================] - 

In [ ]:
combined_history_df = pd.DataFrame(combined_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
combined_history_df.to_csv(f'{output_folder_path}combined_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
combined_autoencoder_model.save(f'{output_folder_path}combined_autoencoder_model.keras')

### Combined Shuffled Training

In [ ]:
# combined Shuffled Model
combined_shuffled_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
combined_shuffled_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_combined_y_train]
)

combined_shuffled_history = combined_shuffled_autoencoder_model.fit(
  x=x_train_combined,
  y=y_train_shuffled,
  batch_size=1,
  epochs=20,
  validation_data = (x_test_combined, y_test_combined)
)

Epoch 1/20
505/505 [==============================] - 4s 6ms/step - loss: -0.7642 - metric_choose_argument_combined_y_train: 0.0000e+00 - val_loss: -0.7758 - val_metric_choose_argument_combined_y_train: 0.0000e+00
Epoch 2/20
505/505 [==============================] - 3s 5ms/step - loss: -0.7774 - metric_choose_argument_combined_y_train: 0.0020 - val_loss: -0.7769 - val_metric_choose_argument_combined_y_train: 0.0000e+00
Epoch 3/20
505/505 [==============================] - 3s 5ms/step - loss: -0.7810 - metric_choose_argument_combined_y_train: 0.0020 - val_loss: -0.7732 - val_metric_choose_argument_combined_y_train: 0.0000e+00
Epoch 4/20
505/505 [==============================] - 3s 5ms/step - loss: -0.7872 - metric_choose_argument_combined_y_train: 0.0158 - val_loss: -0.7726 - val_metric_choose_argument_combined_y_train: 0.0000e+00
Epoch 5/20
505/505 [==============================] - 3s 5ms/step - loss: -0.7979 - metric_choose_argument_combined_y_train: 0.0594 - val_loss: -0.7647 - va

In [ ]:
combined_shuffled_history_df = pd.DataFrame(combined_shuffled_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
combined_shuffled_history_df.to_csv(f'{output_folder_path}combined_shuffled_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
combined_shuffled_autoencoder_model.save(f'{output_folder_path}combined_shuffled_autoencoder_model.keras')

### Pro Training

In [ ]:
# pro Model
pro_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
pro_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_pro_y_train]
)

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='pro_autoencoder_weights.keras', save_best_only=False, save_weights_only=False, verbose=1)
csv_logger_callback = CSVLogger(filename='pro_training_log.csv', separator=',', append=True)
pro_history = pro_autoencoder_model.fit(
  x=x_train_pro,
  y=y_train_pro,
  batch_size=1,
  epochs=20,
  validation_data = (x_test_pro, y_test_pro),
  callbacks=[checkpoint_callback, csv_logger_callback]
)

Epoch 1/20
328/332 [============================>.] - ETA: 0s - loss: -0.7733 - metric_choose_argument_pro_y_train: 0.0061
Epoch 1: saving model to pro_autoencoder_weights.keras
332/332 [==============================] - 3s 6ms/step - loss: -0.7738 - metric_choose_argument_pro_y_train: 0.0060 - val_loss: -0.7952 - val_metric_choose_argument_pro_y_train: 0.0000e+00
Epoch 2/20
324/332 [============================>.] - ETA: 0s - loss: -0.8120 - metric_choose_argument_pro_y_train: 0.0772
Epoch 2: saving model to pro_autoencoder_weights.keras
332/332 [==============================] - 2s 5ms/step - loss: -0.8125 - metric_choose_argument_pro_y_train: 0.0783 - val_loss: -0.8086 - val_metric_choose_argument_pro_y_train: 0.1048
Epoch 3/20
323/332 [============================>.] - ETA: 0s - loss: -0.8392 - metric_choose_argument_pro_y_train: 0.3715
Epoch 3: saving model to pro_autoencoder_weights.keras
332/332 [==============================] - 2s 5ms/step - loss: -0.8391 - metric_choose_argum

In [ ]:
pro_history_df = pd.DataFrame(pro_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
pro_history_df.to_csv(f'{output_folder_path}pro_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
pro_autoencoder_model.save(f'{output_folder_path}pro_autoencoder_model.keras')

### Pro Shuffled Training

In [ ]:
# pro Shuffled Model
pro_shuffled_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
pro_shuffled_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_pro_y_train]
)

pro_shuffled_history = pro_shuffled_autoencoder_model.fit(
  x=x_train_pro,
  y=y_train_pro_shuffled,
  batch_size=1,
  epochs=20,
  validation_data = (x_test_pro, y_test_pro)
)

Epoch 1/20
332/332 [==============================] - 3s 6ms/step - loss: -0.7634 - metric_choose_argument_pro_y_train: 0.0000e+00 - val_loss: -0.7729 - val_metric_choose_argument_pro_y_train: 0.0000e+00
Epoch 2/20
332/332 [==============================] - 2s 5ms/step - loss: -0.7803 - metric_choose_argument_pro_y_train: 0.0030 - val_loss: -0.7735 - val_metric_choose_argument_pro_y_train: 0.0000e+00
Epoch 3/20
332/332 [==============================] - 2s 5ms/step - loss: -0.7852 - metric_choose_argument_pro_y_train: 0.0060 - val_loss: -0.7740 - val_metric_choose_argument_pro_y_train: 0.0000e+00
Epoch 4/20
332/332 [==============================] - 2s 5ms/step - loss: -0.7938 - metric_choose_argument_pro_y_train: 0.0271 - val_loss: -0.7681 - val_metric_choose_argument_pro_y_train: 0.0000e+00
Epoch 5/20
332/332 [==============================] - 2s 5ms/step - loss: -0.8085 - metric_choose_argument_pro_y_train: 0.1657 - val_loss: -0.7532 - val_metric_choose_argument_pro_y_train: 0.0000e

In [ ]:
pro_shuffled_history_df = pd.DataFrame(pro_shuffled_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
pro_shuffled_history_df.to_csv(f'{output_folder_path}pro_shuffled_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
pro_shuffled_autoencoder_model.save(f'{output_folder_path}pro_shuffled_autoencoder_model.keras')

### Counter Training

In [ ]:
# counter Model
counter_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
counter_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_counter_y_train]
)

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='counter_autoencoder_weights.keras', save_best_only=False, save_weights_only=False, verbose=1)
csv_logger_callback = CSVLogger(filename='counter_training_log.csv', separator=',', append=True)
counter_history = counter_autoencoder_model.fit(
  x=x_train_counter,
  y=y_train_counter,
  batch_size=1,
  epochs=20,
  validation_data = (x_test_counter, y_test_counter),
  callbacks=[checkpoint_callback, csv_logger_callback]
)

Epoch 1/20
162/173 [===========================>..] - ETA: 0s - loss: -0.7517 - metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 1: saving model to counter_autoencoder_weights.keras
173/173 [==============================] - 2s 7ms/step - loss: -0.7531 - metric_choose_argument_counter_y_train: 0.0000e+00 - val_loss: -0.7868 - val_metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 2/20
167/173 [===========================>..] - ETA: 0s - loss: -0.8037 - metric_choose_argument_counter_y_train: 0.0599
Epoch 2: saving model to counter_autoencoder_weights.keras
173/173 [==============================] - 1s 6ms/step - loss: -0.8030 - metric_choose_argument_counter_y_train: 0.0578 - val_loss: -0.7942 - val_metric_choose_argument_counter_y_train: 0.0469
Epoch 3/20
164/173 [===========================>..] - ETA: 0s - loss: -0.8333 - metric_choose_argument_counter_y_train: 0.3720
Epoch 3: saving model to counter_autoencoder_weights.keras
173/173 [==============================] - 1

In [ ]:
counter_history_df = pd.DataFrame(counter_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
counter_history_df.to_csv(f'{output_folder_path}counter_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
counter_autoencoder_model.save(f'{output_folder_path}counter_autoencoder_model.keras')

### Counter Shuffled Training

In [ ]:
# counter Shuffled Model
counter_shuffled_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
counter_shuffled_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_counter_y_train]
)

counter_shuffled_history = counter_shuffled_autoencoder_model.fit(
  x=x_train_counter,
  y=y_train_counter_shuffled,
  batch_size=1,
  epochs=20,
  validation_data = (x_test_counter, y_test_counter)
)

Epoch 1/20
173/173 [==============================] - 2s 7ms/step - loss: -0.7455 - metric_choose_argument_counter_y_train: 0.0000e+00 - val_loss: -0.7637 - val_metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 2/20
173/173 [==============================] - 1s 5ms/step - loss: -0.7728 - metric_choose_argument_counter_y_train: 0.0116 - val_loss: -0.7696 - val_metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 3/20
173/173 [==============================] - 1s 5ms/step - loss: -0.7824 - metric_choose_argument_counter_y_train: 0.0173 - val_loss: -0.7690 - val_metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 4/20
173/173 [==============================] - 1s 5ms/step - loss: -0.7970 - metric_choose_argument_counter_y_train: 0.1445 - val_loss: -0.7555 - val_metric_choose_argument_counter_y_train: 0.0000e+00
Epoch 5/20
173/173 [==============================] - 1s 5ms/step - loss: -0.8241 - metric_choose_argument_counter_y_train: 0.4855 - val_loss: -0.7398 - val_metric_

In [ ]:
counter_shuffled_history_df = pd.DataFrame(counter_shuffled_history.history)

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
counter_shuffled_history_df.to_csv(f'{output_folder_path}counter_shuffled_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/nomic-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
counter_shuffled_autoencoder_model.save(f'{output_folder_path}counter_shuffled_autoencoder_model.keras')

## Export data

In [ ]:
def export_nomic_autoencoder():
  """Export nomic_autoencoder directory"""
  nomic_autoencoder_file_path = 'current-data-dump/nomic-autoencoder'
  nomic_autoencoder_file_path_zip = 'current-data-dump/nomic-autoencoder'
  shutil.make_archive(nomic_autoencoder_file_path_zip, 'zip', nomic_autoencoder_file_path)
  print(f"Zip file created at: {nomic_autoencoder_file_path_zip}")
  result = subprocess.run([f"osf -p sakjg upload --force {nomic_autoencoder_file_path_zip}.zip data-dump/nomic-scifact-autoencoder/nomic_scifact_autoencoder.zip"], shell=True, capture_output=True, text=True)
  print(result.stderr)
  print(f"File: {nomic_autoencoder_file_path_zip} uploaded at osfstorage")

In [ ]:
export_nomic_autoencoder()

Zip file created at: current-data-dump/nomic-autoencoder

File: current-data-dump/nomic-autoencoder uploaded at osfstorage


## Import data

In [ ]:
def import_nomic_autoencoder():
  """Import nomic_autoencoder directory"""
  subprocess.run("osf -p sakjg fetch --force osfstorage/data-dump/nomic-scifact-autoencoder/nomic_scifact_autoencoder.zip", shell=True)
  print("nomic_autoencoder.zip successfully imported")
  nomic_autoencoder_file_path_zip = 'nomic_scifact_autoencoder.zip'
  nomic_autoencoder_file_path = 'current-data-dump/nomic-autoencoder'
  os.makedirs(nomic_autoencoder_file_path, exist_ok=True)
  with zipfile.ZipFile(nomic_autoencoder_file_path_zip, 'r') as zip_ref:
    zip_ref.extractall(nomic_autoencoder_file_path)
  extracted_files = os.listdir(nomic_autoencoder_file_path)
  print("Files extracted:", extracted_files)

## Load global training history

In [ ]:
import_nomic_autoencoder()

nomic_autoencoder.zip successfully imported
Files extracted: ['combined_shuffled_training_log.csv', 'pro_training_log.csv', 'counter_training_log.csv', 'counter_training_plot.png', 'x_test_pro.pkl', 'y_train_combined.pkl', 'counter_shuffled_training_log.csv', 'pro_shuffled_training_plot.png', 'y_test_combined.pkl', 'y_test_pro.pkl', 'combined_shuffled_training_plot.png', 'pro_shuffled_autoencoder_model.keras', 'combined_autoencoder_model.keras', 'pro_training_plot.png', 'y_train_counter.pkl', 'y_test_counter.pkl', 'y_train_pro.pkl', 'x_train_combined.pkl', 'combined_training_plot.png', 'all_pro_training_plot.png', 'x_train_counter.pkl', 'counter_shuffled_autoencoder_model.keras', 'x_test_combined.pkl', 'counter_shuffled_training_plot.png', 'counter_autoencoder_model.keras', 'all_combined_training_plot.png', 'x_test_counter.pkl', 'pro_autoencoder_model.keras', 'combined_shuffled_autoencoder_model.keras', 'all_counter_training_plot.png', 'combined_training_log.csv', 'x_train_pro.pkl', 'p

#### Unshuffled training history

In [ ]:
# Access training history
loaded_combined_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/combined_training_log.csv"))
loaded_combined_history = pd.melt(loaded_combined_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_combined_y_train', 'val_metric_choose_argument_combined_y_train'], var_name='dataset', value_name='accuracy')
loaded_combined_history = loaded_combined_history.replace(['metric_choose_argument_combined_y_train', 'val_metric_choose_argument_combined_y_train'], ['training set', 'validation set'])
loaded_combined_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_combined_history['shuffled'] = False

loaded_pro_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/pro_training_log.csv"))
loaded_pro_history = pd.melt(loaded_pro_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_pro_y_train', 'val_metric_choose_argument_pro_y_train'], var_name='dataset', value_name='accuracy')
loaded_pro_history = loaded_pro_history.replace(['metric_choose_argument_pro_y_train', 'val_metric_choose_argument_pro_y_train'], ['training set', 'validation set'])
loaded_pro_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_pro_history['shuffled'] = False

loaded_counter_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/counter_training_log.csv"))
loaded_counter_history = pd.melt(loaded_counter_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_counter_y_train', 'val_metric_choose_argument_counter_y_train'], var_name='dataset', value_name='accuracy')
loaded_counter_history = loaded_counter_history.replace(['metric_choose_argument_counter_y_train', 'val_metric_choose_argument_counter_y_train'], ['training set', 'validation set'])
loaded_counter_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_counter_history['shuffled'] = False

In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

combined_training_plot = (
    ggplot(loaded_combined_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Unshuffled Data', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(combined_training_plot, "current-data-dump/nomic-autoencoder/combined_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/combined_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

pro_training_plot = (
    ggplot(loaded_pro_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Unshuffled Data (Pro-evidence Only)', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(pro_training_plot, "current-data-dump/nomic-autoencoder/pro_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/pro_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

counter_training_plot = (
    ggplot(loaded_counter_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Unshuffled Data (Counterevidence Only)', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(counter_training_plot, "current-data-dump/nomic-autoencoder/counter_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/counter_training_plot.png


#### Shuffled training history

In [ ]:
# Access training history
loaded_combined_shuffled_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/combined_shuffled_training_log.csv"))
loaded_combined_shuffled_history = pd.melt(loaded_combined_shuffled_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_combined_y_train', 'val_metric_choose_argument_combined_y_train'], var_name='dataset', value_name='accuracy')
loaded_combined_shuffled_history = loaded_combined_shuffled_history.replace(['metric_choose_argument_combined_y_train', 'val_metric_choose_argument_combined_y_train'], ['training set', 'validation set'])
loaded_combined_shuffled_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_combined_shuffled_history['shuffled'] = True

loaded_pro_shuffled_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/pro_shuffled_training_log.csv"))
loaded_pro_shuffled_history = pd.melt(loaded_pro_shuffled_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_pro_y_train', 'val_metric_choose_argument_pro_y_train'], var_name='dataset', value_name='accuracy')
loaded_pro_shuffled_history = loaded_pro_shuffled_history.replace(['metric_choose_argument_pro_y_train', 'val_metric_choose_argument_pro_y_train'], ['training set', 'validation set'])
loaded_pro_shuffled_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_pro_shuffled_history['shuffled'] = True

loaded_counter_shuffled_history = pd.DataFrame(pd.read_csv("current-data-dump/nomic-autoencoder/counter_shuffled_training_log.csv"))
loaded_counter_shuffled_history = pd.melt(loaded_counter_shuffled_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_counter_y_train', 'val_metric_choose_argument_counter_y_train'], var_name='dataset', value_name='accuracy')
loaded_counter_shuffled_history = loaded_counter_shuffled_history.replace(['metric_choose_argument_counter_y_train', 'val_metric_choose_argument_counter_y_train'], ['training set', 'validation set'])
loaded_counter_shuffled_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_counter_shuffled_history['shuffled'] = True

In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

combined_shuffled_training_plot = (
    ggplot(loaded_combined_shuffled_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Shuffled Data', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(combined_shuffled_training_plot, "current-data-dump/nomic-autoencoder/combined_shuffled_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/combined_shuffled_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

pro_shuffled_training_plot = (
    ggplot(loaded_pro_shuffled_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Shuffled Data (Pro-evidence Only)', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(pro_shuffled_training_plot, "current-data-dump/nomic-autoencoder/pro_shuffled_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/pro_shuffled_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

counter_shuffled_training_plot = (
    ggplot(loaded_counter_shuffled_history, aes(x='epoch', y='accuracy', linetype='dataset')) +
    geom_line(size=2) +
    labs(title='Learning Curve of Model Trained on Shuffled Data (Counterevidence Only)', x='Epoch', y='Accuracy') +
    scale_y_continuous(limits=[0,1]) +
    scale_linetype_manual(linetype_map) +
    theme(
        plot_title=element_text(size=40, lineheight=1.5, wrap=True),
        axis_title=element_text(size=32),
        axis_text=element_text(size=24),
        legend_position='bottom',
        legend_text=element_text(size=24),
        legend_title=element_text(size=32),
        figure_size=[16,24]
        )
)
ggsave(counter_shuffled_training_plot, "current-data-dump/nomic-autoencoder/counter_shuffled_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/counter_shuffled_training_plot.png


## Combined Training Plots

In [ ]:
all_combined_training_df = pd.concat([loaded_combined_history, loaded_combined_shuffled_history])
all_pro_training_df = pd.concat([loaded_pro_history, loaded_pro_shuffled_history])
all_counter_training_df = pd.concat([loaded_counter_history, loaded_counter_shuffled_history])

In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

all_combined_plot = (
  ggplot(all_combined_training_df, aes(x='epoch', y='accuracy', linetype='dataset', color='shuffled')) +
  geom_line(size=2) +
  labs(title='Learning Curve of Model Trained on Unshuffled vs. Within-Topic Shuffled Data', x='Epoch', y='Accuracy') +
  scale_linetype_manual(linetype_map) +
  theme(
    figure_size=(16,24),
    axis_title=element_text(size=32),
    axis_text=element_text(size=24),
    legend_title=element_text(size=32, lineheight=1.5),
    legend_text=element_text(size=24, lineheight=1.5),
    plot_title=element_text(size=40, wrap=True, lineheight=1.5),
    legend_position="bottom",
    legend_key_width=64
  ) +
  guides(fill = guide_legend(byrow = True))
)
ggsave(all_combined_plot, "current-data-dump/nomic-autoencoder/all_combined_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/all_combined_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

all_pro_plot = (
  ggplot(all_pro_training_df, aes(x='epoch', y='accuracy', linetype='dataset', color='shuffled')) +
  geom_line(size=2) +
  labs(title='Learning Curve of Model Trained on Unshuffled vs. Within-Topic Shuffled Data (Pro-evidence Only)', x='Epoch', y='Accuracy') +
  scale_linetype_manual(linetype_map) +
  theme(
    figure_size=(16,24),
    axis_title=element_text(size=32),
    axis_text=element_text(size=24),
    legend_title=element_text(size=32, lineheight=1.5),
    legend_text=element_text(size=24, lineheight=1.5),
    plot_title=element_text(size=40, wrap=True, lineheight=1.5),
    legend_position="bottom",
    legend_key_width=64
  ) +
  guides(fill = guide_legend(byrow = True))
)
ggsave(all_pro_plot, "current-data-dump/nomic-autoencoder/all_pro_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/all_pro_training_plot.png


In [ ]:
linetype_map={'validation set': 'solid', 'training set': 'dashed'}

all_counter_plot = (
  ggplot(all_counter_training_df, aes(x='epoch', y='accuracy', linetype='dataset', color='shuffled')) +
  geom_line(size=2) +
  labs(title='Learning Curve of Model Trained on Unshuffled vs. Within-Topic Shuffled Data (Counterevidence Only)', x='Epoch', y='Accuracy') +
  scale_linetype_manual(linetype_map) +
  theme(
    figure_size=(16,24),
    axis_title=element_text(size=32),
    axis_text=element_text(size=24),
    legend_title=element_text(size=32, lineheight=1.5),
    legend_text=element_text(size=24, lineheight=1.5),
    plot_title=element_text(size=40, wrap=True, lineheight=1.5),
    legend_position="bottom",
    legend_key_width=64
  ) +
  guides(fill = guide_legend(byrow = True))
)
ggsave(all_counter_plot, "current-data-dump/nomic-autoencoder/all_counter_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: current-data-dump/nomic-autoencoder/all_counter_training_plot.png


## Final Export

In [ ]:
export_nomic_autoencoder()

NameError: name 'export_nomic_autoencoder' is not defined